In [7]:
import pyspark
from pyspark.sql import SparkSession
import os

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")
NESSIE_URI = os.getenv("NESSIE_URI")


def create_spark_session(appname):
    master = "spark://spark-master:7077"

    conf = (
        pyspark.SparkConf()
        .setAppName(appname)
        .set("spark.master", master)

        # EXTENSÕES ICEBERG + NESSIE
        .set(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )

        # REGISTRO DO CATÁLOGO NESSIE
        .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.nessie.type", "nessie")
        .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.nessie.uri", NESSIE_URI)
        .set("spark.sql.catalog.nessie.ref", "main")
        .set("spark.sql.catalog.nessie.authentication.type", "NONE")
        .set("spark.sql.catalog.nessie.cache-enabled", "false")
        .set("spark.sql.catalog.nessie.warehouse", "s3a://lakehouse/warehouse/")

        # CONFIG S3 -> ICEBERG
        .set("spark.sql.catalog.nessie.s3.path-style-access", "true")
        .set("spark.sql.catalog.nessie.s3.endpoint", S3_ENDPOINT)

        # CONFIG HADOOP S3A
        .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
        .set("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
        .set("spark.hadoop.fs.s3a.path.style.access", "true")
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .set(
                "spark.hadoop.fs.s3a.aws.credentials.provider",
                "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"
            )

        # RECURSOS
        .set("spark.executor.memory", "1g")
        .set("spark.executor.cores", "1")
        .set("spark.driver.memory", "1g")
        .set("spark.executor.instances", "1")
    )

    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    return spark



In [15]:
spark.stop()

In [2]:
spark = create_spark_session("bronze")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/24 00:11:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [11]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [4]:
from pyspark.sql import functions as F
from datetime import datetime
from uuid import uuid4
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


class Bronze:
    """
    Classe responsável pela camada Bronze do Lakehouse Tibia.

    A classe realiza:
    - Configuração do catálogo Nessie + Iceberg.
    - Criação dos namespaces e tabelas caso não existam.
    - Leitura dos arquivos CSV da camada Landing.
    - Padronização dos dados.
    - Registro de metadados operacionais.
    - Escrita incremental via Iceberg (append).

    Parameters
    ----------
    spark : SparkSession
        Sessão Spark ativa criada no job.
    date_str : str | None
        Data fornecida via CLI no formato 'YYYY-MM-DD'.
        Caso None, utiliza a data atual para construir o path da camada Landing.
    """

    def __init__(self, spark, date_str):
        self.spark = spark
        self.date_str = date_str

    # ============================================================
    #   MÉTODO: VOCATION
    # ============================================================
    def vocation(self):
        """
        Executa o job Bronze para dados de vocação (vocation).

        1. Configura catálogo Iceberg + warehouse.
        2. Cria namespace + tabela caso não existam.
        3. Lê arquivos CSV da camada landing.
        4. Valida colunas obrigatórias.
        5. Normaliza nomes e tipos de colunas.
        6. Gera batch_id e colunas de auditoria.
        7. Deduplica por (name, world).
        8. Insere incrementalmente na tabela Bronze.

        Tabela criada: nessie.bronze.vocation
        """

        # Namespace
        self.spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.bronze")

        # Criação da tabela Iceberg
        self.spark.sql("""
        CREATE TABLE IF NOT EXISTS nessie.bronze.vocation (
            name STRING,
            vocation STRING,
            level INT,
            world STRING,
            experience LONG,
            world_type STRING,
            ingestion_time TIMESTAMP,
            ingestion_date DATE,
            source_system STRING,
            batch_id STRING
        )
        USING iceberg
        PARTITIONED BY (world, ingestion_date)
        TBLPROPERTIES (
            'format-version' = '2',
            'write.format.default' = 'parquet',
            'write.metadata.compression' = 'gzip',
            'write.delete.mode' = 'merge-on-read'
        )
        """)

        # Define path da landing com base na data
        today_date = datetime.strptime(self.date_str, "%Y-%m-%d") if self.date_str else datetime.today()
        partition = f"year={today_date.year}/month={today_date.month}/day={today_date.day}"

        path = f"s3a://landing/{partition}/vocation/*.csv"
        logging.info(f"Lendo dados de: {path}")

        # Lê arquivo CSV
        df_raw = self.spark.read.csv(path, header=True)

        # Validação de colunas esperadas
        colunas_esperadas = {"Rank", "Name", "Vocation", "World", "Level", "Points", "WorldType"}
        colunas_faltando = colunas_esperadas - set(df_raw.columns)

        if colunas_faltando:
            logging.error(f"Colunas ausentes no CSV: {colunas_faltando}")
            return

        # Gera batch_id para auditoria
        batch_id = str(uuid4())
        logging.info(f"Gerando batch_id: {batch_id}")

        df_raw.printSchema()

        # Normalização e padronização
        df_bronze = (
            df_raw.drop("Rank")
            .withColumnRenamed("Name", "name")
            .withColumnRenamed("Vocation", "vocation")
            .withColumnRenamed("Level", "level")
            .withColumnRenamed("World", "world")
            .withColumnRenamed("Points", "experience")
            .withColumnRenamed("WorldType", "world_type")
            .withColumn("ingestion_time", F.current_timestamp())
            .withColumn("ingestion_date", F.current_date())
            .withColumn("source_system", F.lit("highscore_tibia_page"))
            .withColumn("batch_id", F.lit(batch_id))
            .withColumn("experience", F.regexp_replace("experience", ",", "").cast("long"))
            .withColumn("level", F.col("level").cast("int"))
            .withColumn("vocation", F.trim(F.lower("vocation")))
            .withColumn("world", F.trim(F.lower("world")))
            .dropDuplicates(["name", "world"])
        )

        # Compressão padrão Parquet
        self.spark.conf.set("spark.sql.parquet.compression.codec", "snappy")

        record_count = df_bronze.count()

        if record_count > 0:
            logging.info(f"Inserindo {record_count} registros na Bronze com batch_id {batch_id}...")
            df_bronze.writeTo("nessie.bronze.vocation").append()
        else:
            logging.warning("Nenhum registro encontrado para gravar na Bronze.")

In [5]:
bronze = Bronze(spark, "2025-07-12")

In [6]:
bronze.vocation()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Py4JJavaError: An error occurred while calling o67.sql.
: software.amazon.awssdk.services.s3.model.NoSuchBucketException: The specified bucket does not exist (Service: S3, Status Code: 404, Request ID: 1883FE542BF71687, Extended Request ID: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleErrorResponse(AwsXmlPredicatedResponseHandler.java:156)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handleResponse(AwsXmlPredicatedResponseHandler.java:108)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handle(AwsXmlPredicatedResponseHandler.java:85)
	at software.amazon.awssdk.protocols.xml.internal.unmarshall.AwsXmlPredicatedResponseHandler.handle(AwsXmlPredicatedResponseHandler.java:43)
	at software.amazon.awssdk.awscore.client.handler.AwsSyncClientHandler$Crc32ValidationResponseHandler.hand

In [7]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.bronze.vocation")

        # # Criação da tabela Iceberg
        # self.spark.sql("""
        # CREATE TABLE IF NOT EXISTS nessie.bronze.vocation (
        #     name STRING,
        #     vocation STRING,
        #     level INT,
        #     world STRING,
        #     experience LONG,
        #     world_type STRING,
        #     ingestion_time TIMESTAMP,
        #     ingestion_date DATE,
        #     source_system STRING,
        #     batch_id STRING
        # )
        # USING iceberg
        # PARTITIONED BY (world, ingestion_date)
        # TBLPROPERTIES (
        #     'format-version' = '2',
        #     'write.format.default' = 'parquet',
        #     'write.metadata.compression' = 'gzip',
        #     'write.delete.mode' = 'merge-on-read'
        # )
        # """)

Py4JJavaError: An error occurred while calling o71.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'nessie': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1926)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:54)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:54)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:30)
	at org